# Regression and Classification Using Tensorflow

Tensorflow can be used for both linear regression and classification

### Linear Regression using tensorflow

In [1]:
# We start by loading the necessary libraries, creating a graph, and loading the data:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import tensorflow as tf
from sklearn import datasets
from tensorflow.python.framework import ops
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

ops.reset_default_graph()
sess = tf.Session()

In [3]:
train = pd.read_csv('../datasets/housing-prices/train.csv')
test = pd.read_csv('../datasets/housing-prices/test.csv')
testID = test['Id']

y = np.log1p(train['SalePrice'].values)
data = pd.concat([train.drop('SalePrice', axis=1), test], keys=['train', 'test'])
data.drop(['Id'], axis=1, inplace=True)

In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
data['MSSubClass'] = data['MSSubClass'].map(lambda x: 'MSSubClass_'+str(x))

In [5]:
MSSubClass = pd.get_dummies(data['MSSubClass'])

In [6]:
MSZoning = pd.get_dummies(data['MSZoning'])
MSZoning.shape

(2919, 5)

In [7]:
data['LotFrontage'].fillna(np.nanmedian(data['LotFrontage']), inplace=True)

In [8]:
bins = [0, 25, 30, 40, 50, 60, 70, 80, 100, 150, 200, 250, 350]
LotFrontage_bins = pd.get_dummies(pd.cut(data['LotFrontage'], bins))
LotFrontage_bins.shape

(2919, 12)

In [9]:
from sklearn.preprocessing import StandardScaler
X = np.log10(data["LotArea"]).values
print(X.shape)
             
LotArea = StandardScaler().fit_transform(X.reshape(-1,1))
print(LotArea.shape)

(2919,)
(2919, 1)


In [10]:
Street = pd.get_dummies(data['Street'])
Street.shape

(2919, 2)

In [11]:
LotShape = pd.get_dummies(data['LotShape'])
LotShape.shape

(2919, 4)

In [12]:
data['LandContour'].value_counts(dropna=False)

Lvl    2622
HLS    120 
Bnk    117 
Low    60  
Name: LandContour, dtype: int64

In [13]:
LandContour = pd.get_dummies(data['LandContour'])
LandContour.shape

(2919, 4)

In [14]:
num_cols = ['LotFrontage', 'LotArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
           '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
           'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF',
           'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea']

cat_cols = [col for col in data.columns.tolist() if col not in num_cols]

In [15]:
len(num_cols), len(cat_cols)

(24, 55)

In [16]:
from tqdm import tqdm_notebook
data_cat = pd.DataFrame()
for col in tqdm_notebook(cat_cols):
    if data[col].isnull().sum() > 0:
        data[col].fillna('UNK_'+col, inplace=True)
    data[col] = data[col].map(lambda x: col+str(x))
    mydf = pd.get_dummies(data[col])
    if len(data_cat) == 0:
        data_cat = mydf
    else:
        data_cat = pd.concat([data_cat, mydf], axis=1)

In [17]:
bins = [0, 25, 30, 40, 50, 60, 70, 80, 100, 150, 200, 250, 350]
data['LotFrontage'].fillna(np.nanmedian(data['LotFrontage']), inplace=True)
columns = ['LotFrontage_bin'+str(x) for x in range(len(bins)-1)]
LotFrontage_bins = pd.get_dummies(pd.cut(data['LotFrontage'], bins))
LotFrontage_bins.columns = columns
data_cat = pd.concat([data_cat, LotFrontage_bins], axis=1)

In [18]:
num_cols.remove('LotFrontage')

In [19]:
data_num = data[num_cols]

In [20]:
for col in num_cols:
    print(col, data_num[col].nunique(dropna=False))
    data_num[col].fillna(np.nanmedian(data_num[col]))

LotArea 1951
BsmtFinSF1 992
BsmtFinSF2 273
BsmtUnfSF 1136
TotalBsmtSF 1059
1stFlrSF 1083
2ndFlrSF 635
LowQualFinSF 36
GrLivArea 1292
BsmtFullBath 5
BsmtHalfBath 4
FullBath 5
HalfBath 3
TotRmsAbvGrd 14
Fireplaces 5
GarageCars 7
GarageArea 604
WoodDeckSF 379
OpenPorchSF 252
EnclosedPorch 183
3SsnPorch 31
ScreenPorch 121
PoolArea 14


In [21]:
from sklearn.preprocessing import StandardScaler
for col in num_cols:
    print(col)
    if data_num[col].isnull().any() == True:
        print(col, np.nanmedian(data_num[col]))
        data_num[col].fillna(np.nanmedian(data_num[col]), inplace=True)
    X = data_num[col].values
    data_num[col] = StandardScaler().fit_transform(X.reshape(-1,1)).ravel()

LotArea
BsmtFinSF1
BsmtFinSF1 368.5


/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame



BsmtFinSF2
BsmtFinSF2 0.0
BsmtUnfSF
BsmtUnfSF 467.0


/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

TotalBsmtSF
TotalBsmtSF 989.5
1stFlrSF
2ndFlrSF


...

/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

GarageCars
GarageCars 2.0
GarageArea
GarageArea 480.0
WoodDeckSF


/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea


/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/watts/anaconda3/envs/itv/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardS

In [22]:
data_num.shape, data_cat.shape

((2919, 23), (2919, 1117))

In [23]:
data_processed = pd.concat([data_num, data_cat], axis=1)

In [24]:
data_processed.shape

(2919, 1140)

In [25]:
y

array([12.24769912, 12.10901644, 12.31717117, ..., 12.49313327,
       11.86446927, 11.90159023])

In [26]:
from sklearn.model_selection import train_test_split

x_vals = data_processed.values

x_vals_train = x_vals[:len(train)]
x_vals_test = x_vals[len(train):]

X = x_vals_train
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
x_vals.shape,y.shape

((2919, 1140), (1460,))

In [28]:
X_train.shape,y.shape, X_val.shape

((1168, 1140), (1460,), (292, 1140))

In [29]:
np.save('../cache/X_train', X_train)
np.save('../cache/X_val', X_val)
np.save('../cache/y_train', y_train)
np.save('../cache/y_val', y_val)

In [29]:
# We declare our learning rate, batch size, placeholders, and model variables:
learning_rate = 0.0005
batch_size = 25
x_data = tf.placeholder(shape=[None,X_train.shape[1]], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
A = tf.Variable(tf.random_normal(shape=[X_train.shape[1], 1], mean=0, stddev=1, dtype=tf.float32))
b = tf.Variable(tf.random_normal(shape=[1,1], mean=0, stddev=1, dtype=tf.float32))

In [30]:
# learning_rate = 0.05
# batch_size = 25
# x_data = tf.placeholder(shape=[None, 1], dtype=tf.float32)
# y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
# A = tf.Variable(tf.random_normal(shape=[1,1]))
# b = tf.Variable(tf.random_normal(shape=[1,1]))

In [31]:
# Next, we write the formula for the linear model, y=Ax+b:
model_output = tf.add(tf.matmul(x_data, A), b)

In [32]:
# Then we declare our L2 loss function (which includes the mean over the batch),
# initialize the variables, and declare our optimizer. Note that we chose 0.05 as our
# learning rate:
loss = tf.reduce_mean(tf.square(y_target - model_output))

init = tf.global_variables_initializer()
sess.run(init)
my_opt = tf.train.GradientDescentOptimizer(learning_rate)
train_step = my_opt.minimize(loss)

In [33]:
print(sess.run(A))

[[ 1.1214571 ]
 [-0.8559874 ]
 [ 1.6588043 ]
 ...
 [-0.65630585]
 [-0.4111823 ]
 [-0.2326453 ]]


In [34]:
print(sess.run(b))

[[0.33805653]]


In [35]:
X_train

array([[-0.22421933,  1.05520839, -0.29302528, ...,  0.        ,
         0.        ,  0.        ],
       [-0.29561488, -0.96913327, -0.29302528, ...,  0.        ,
         0.        ,  0.        ],
       [-0.17641094, -0.96913327, -0.29302528, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.25313262, -0.60246835, -0.29302528, ...,  0.        ,
         0.        ,  0.        ],
       [-0.32034336, -0.96913327, -0.29302528, ...,  0.        ,
         0.        ,  0.        ],
       [-0.82226804, -0.96913327, -0.29302528, ...,  0.        ,
         0.        ,  0.        ]])

In [36]:
# We can now loop through and train the model on randomly selected batches. We will
# run it for 100 loops and print out the variable and loss values every 25 iterations.
# Note that here we are also saving the loss of every iteration so that we can view it
# afterwards:
loss_vec = []
for i in tqdm_notebook(range(30000)):
    rand_index = np.random.choice(len(X_train), size=batch_size)
    rand_x = X_train[rand_index]
    rand_y = y_train[rand_index].reshape(-1,1)
    sess.run(train_step, feed_dict={x_data: rand_x, y_target:rand_y})
    temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
    loss_vec.append(temp_loss)
    if (i+1)%1000==0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)) + 'b = ' + str(sess.run(b)))
        print('Loss = ''' + str(temp_loss))

Step #1000 A = [[ 0.48974478]
 [-0.12419165]
 [ 0.82702595]
 ...
 [-0.6324297 ]
 [-0.4111823 ]
 [-0.21832767]]b = [[0.9624538]]
Loss = 20.910908
Step #2000 A = [[ 0.3515749 ]
 [ 0.03470074]
 [ 0.4577966 ]
 ...
 [-0.6228576 ]
 [-0.4111823 ]
 [-0.20873918]]b = [[1.0075605]]
Loss = 17.824255
Step #3000 A = [[ 0.24128829]
 [ 0.06589257]
 [ 0.31792694]
 ...
 [-0.6213335 ]
 [-0.4111823 ]
 [-0.20007278]]b = [[1.0294656]]
Loss = 9.725162
Step #4000 A = [[ 0.19500169]
 [ 0.07976186]
 [ 0.2599054 ]
 ...
 [-0.6194118 ]
 [-0.4111823 ]
 [-0.19048332]]b = [[1.0541595]]
Loss = 6.665202
Step #5000 A = [[ 0.15584561]
 [ 0.08072476]
 [ 0.2442012 ]
 ...
 [-0.6169766 ]
 [-0.4111823 ]
 [-0.18112098]]b = [[1.0748761]]
Loss = 10.466142
Step #6000 A = [[ 0.15058102]
 [ 0.08054748]
 [ 0.23341753]
 ...
 [-0.6130693 ]
 [-0.4111823 ]
 [-0.17429607]]b = [[1.0850526]]
Loss = 6.5910997
Step #7000 A = [[ 0.16494745]
 [ 0.09234081]
 [ 0.23866084]
 ...
 [-0.6125618 ]
 [-0.4111823 ]
 [-0.16858369]]b = [[1.0949377]]
Loss

In [39]:

X_val = X_val.astype(np.float32)
y_val_predicted = sess.run(tf.matmul(X_val, A) + b)

In [40]:
y_val_predicted.shape

(292, 1)

In [41]:
y_val.shape

(292,)

In [42]:
from sklearn.metrics import r2_score

r2_score(y_val, y_val_predicted)  

-13.854455151883359

### Implementing Lasso Regression

In [1]:
# We start by loading the necessary libraries, creating a graph, and loading the data:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import tensorflow as tf
from sklearn import datasets
from tensorflow.python.framework import ops
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook
from sklearn.metrics import r2_score

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

ops.reset_default_graph()
sess = tf.Session()

In [2]:
X_train = np.load('../cache/X_train.npy')
X_val = np.load('../cache/X_val.npy')
y_train = np.load('../cache/y_train.npy')
y_val = np.load('../cache/y_val.npy')

#### Heaviside Step Function

The Heaviside step function, or the unit step function, usually denoted by H or θ (but sometimes u, 1 or 𝟙), is a discontinuous function, named after Oliver Heaviside (1850–1925), whose value is zero for negative argument and one for positive argument. 

In [3]:
# We add the loss function, which is a modified continuous heavyside step function.
# We also set the cutoff for lasso regression at 0.9 . This means that we want to restrict
# the slope coefficient to be less than 0.9 . Use the following code:
bach_size = 50
x_data = tf.placeholder(shape=[None,X_train.shape[1]], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
reg_param = tf.placeholder(shape=[1,1], dtype=tf.float32)
A = tf.Variable(tf.random_normal(shape=[X_train.shape[1], 1], dtype=tf.float32))
b = tf.Variable(tf.random_normal(shape=[1,1], mean=0, stddev=1, dtype=tf.float32))

# Declare Lasso loss function
# Lasso Loss = L2_Loss + heavyside_step,
# Where heavyside_step ~ 0 if A < constant, otherwise ~ 99
lasso_param = tf.constant(0.9)
heavyside_step = tf.truediv(1., tf.add(1., tf.exp(tf.multiply(-100.,tf.reduce_mean(tf.subtract(A, lasso_param))))))
regularization_param = tf.multiply(heavyside_step, 99.)
# model_output = tf.add(tf.add(tf.matmul(x_data, A), b), regularization_param)
model_output = tf.add(tf.matmul(x_data, A), b)
lasso_loss = tf.add(tf.reduce_mean(tf.square(y_target - model_output)),reg_param)

learning_rate = 0.001
init = tf.global_variables_initializer()
sess.run(init)
my_opt = tf.train.GradientDescentOptimizer(learning_rate)
train_step = my_opt.minimize(lasso_loss)

In [4]:
# sess.run(A)
# sess.run(heavyside_step)
sess.run(regularization_param)

0.0

In [5]:
loss_vec = []
batch_size = 25
for i in tqdm_notebook(range(30000)):
    rand_index = np.random.choice(len(X_train), size=batch_size)
    rand_x = X_train[rand_index]
    rand_y = y_train[rand_index].reshape(-1,1)
    reg_par = sess.run(regularization_param)
    sess.run(train_step, feed_dict={x_data: rand_x, y_target:rand_y })
    temp_loss = sess.run(lasso_loss, feed_dict={x_data: rand_x, y_target: rand_y, reg_param: reg_par.reshape(-1,1)})

    loss_vec.append(temp_loss)
    if (i+1)%10000==0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)) + 'b = ' + str(sess.run(b)))
        print('Loss = ''' + str(temp_loss))

Step #10000 A = [[ 0.01745011]
 [-0.18922137]
 [-0.12209782]
 ...
 [ 0.44511452]
 [-1.1057391 ]
 [-0.6921445 ]]b = [[1.7665231]]
Loss = [[3.3702471]]
Step #20000 A = [[ 0.0148832 ]
 [-0.10084993]
 [-0.07006028]
 ...
 [ 0.4019568 ]
 [-1.1057391 ]
 [-0.69881916]]b = [[1.7931547]]
Loss = [[1.3979422]]
Step #30000 A = [[ 0.0078067 ]
 [-0.06836872]
 [-0.04086526]
 ...
 [ 0.35921973]
 [-1.1057391 ]
 [-0.6989342 ]]b = [[1.8102524]]
Loss = [[1.49942]]



In [6]:
X_val = X_val.astype(np.float32)
y_val_predicted = sess.run(tf.matmul(X_val, A) + b)
r2_score(y_val, y_val_predicted)  

-6.048774706635721

### Ridge Regression using Tensorflow

In [7]:
# We start by loading the necessary libraries, creating a graph, and loading the data:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import tensorflow as tf
from sklearn import datasets
from tensorflow.python.framework import ops
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook
from sklearn.metrics import r2_score

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

ops.reset_default_graph()
sess = tf.Session()

In [8]:
X_train = np.load('../cache/X_train.npy')
X_val = np.load('../cache/X_val.npy')
y_train = np.load('../cache/y_train.npy')
y_val = np.load('../cache/y_val.npy')

In [13]:
# We add the loss function, which is a modified continuous heavyside step function.
# We also set the cutoff for lasso regression at 0.9 . This means that we want to restrict
# the slope coefficient to be less than 0.9 . Use the following code:
bach_size = 50
x_data = tf.placeholder(shape=[None,X_train.shape[1]], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)

A = tf.Variable(tf.random_normal(shape=[X_train.shape[1], 1], dtype=tf.float32))
b = tf.Variable(tf.random_normal(shape=[1,1], mean=0, stddev=1, dtype=tf.float32))
model_output = tf.add(tf.matmul(x_data, A), b)

ridge_param = tf.constant(1.)
ridge_loss = tf.reduce_mean(tf.square(A))
loss = tf.expand_dims(
    tf.add(
        tf.reduce_mean(
            tf.square(y_target -model_output)), 
        tf.multiply(ridge_param, ridge_loss)), 
    0)

learning_rate = 0.001
init = tf.global_variables_initializer()
sess.run(init)
my_opt = tf.train.GradientDescentOptimizer(learning_rate)
train_step = my_opt.minimize(loss)

In [17]:
loss_vec = []
batch_size = 100
for i in tqdm_notebook(range(30000)):
    rand_index = np.random.choice(len(X_train), size=batch_size)
    rand_x = X_train[rand_index]
    rand_y = y_train[rand_index].reshape(-1,1)
    
    sess.run(train_step, feed_dict={x_data: rand_x, y_target:rand_y })
    temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})

    loss_vec.append(temp_loss)
    if (i+1)%10000==0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)) + 'b = ' + str(sess.run(b)))
        print('Loss = ''' + str(temp_loss))

Step #10000 A = [[ 8.3298311e-02]
 [-1.7279442e-01]
 [-7.0724869e-04]
 ...
 [ 1.0518183e+00]
 [ 8.6220197e-02]
 [-6.0501271e-01]]b = [[0.44074133]]
Loss = [1.2208205]
Step #20000 A = [[ 0.08151925]
 [-0.16896461]
 [ 0.00164573]
 ...
 [ 1.0452977 ]
 [ 0.08473008]
 [-0.60186017]]b = [[0.46807453]]
Loss = [1.0673039]
Step #30000 A = [[ 0.08028734]
 [-0.16173564]
 [ 0.00122472]
 ...
 [ 1.0422298 ]
 [ 0.08323997]
 [-0.596484  ]]b = [[0.49489278]]
Loss = [0.97903097]



In [18]:
X_val = X_val.astype(np.float32)
y_val_predicted = sess.run(tf.matmul(X_val, A) + b)
r2_score(y_val, y_val_predicted)  

-4.332496080720992

### Regression using Keras

### Sequential model

The Keras Sequential model is a linear stack of layers.

### Dense layer
Dense implements the operation: output = activation(dot(input, kernel) + bias) where activation is the element-wise activation function passed as the activation argument, kernel is a weights matrix created by the layer, and bias is a bias vector created by the layer (only applicable if use_bias is True).

### Dropout

Simply put, dropout refers to ignoring units (i.e. neurons) during the training phase of certain set of neurons which is chosen at random. By “ignoring”, I mean these units are not considered during a particular forward or backward pass.

why do we need dropout at all? Why do we need to literally shut-down parts of a neural networks?

The answer to these questions is “to prevent over-fitting”.

A fully connected layer occupies most of the parameters, and hence, neurons develop co-dependency amongst each other during training which curbs the individual power of each neuron leading to over-fitting of training data.

In [223]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers
from keras import regularizers

# the number of dense layers and number of dense neurons in by trial and error
model = Sequential()
model.add(Dense(256, input_dim=1140))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('linear')) 

# For a mean squared error regression problem
sgd = optimizers.SGD(lr=0.0035, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='mse', metrics=['mse'])

In [224]:
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=[X_val,y_val], verbose=0)

In [225]:
y_val_predicted = model.predict(X_val)

In [226]:
r2_score(y_val, y_val_predicted)  

0.9003606291118122